Desafio: Construir um estimador de sinais i1D robusto
1- Elaborar uma função para estimar sinais i1D e sua confiança respctiva
2- Crie uma análise em escalas piramidal gaussiana para criar um detector robusto
3- Derive uma metrica para definir robustez da estimativa
4- Explicar res

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from skimage.feature import peak_local_max
from scipy.signal import convolve2d as conv2

In [3]:
def estimate_i1d(image):
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    # Detecta bordas com Canny
    edges = cv.Canny(image, 100, 200)

    # Detecta linhas com Hough
    lines = cv.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength=100, maxLineGap=10)

    # Cria uma cópia da imagem para desenhar as linhas e vetores normais
    image_with_lines_and_normals = cv.cvtColor(image, cv.COLOR_GRAY2BGR)

    # Para cada linha...
    for line in lines:
        x1, y1, x2, y2 = line[0]

        # Desenha a linha
        cv.line(image_with_lines_and_normals, (x1, y1), (x2, y2), (0, 255, 0), 1)

        # Calcula o vetor direção da linha
        vx, vy = x2 - x1, y2 - y1

        # Normaliza o vetor
        mag = np.sqrt(vx**2 + vy**2)
        vx, vy = vx / mag, vy / mag

        # Calcula o vetor normal
        nx, ny = -vy, vx

        # Calcula o ponto médio da linha
        mx, my = (x1 + x2) / 2, (y1 + y2) / 2

        # Calcula o ponto final do vetor normal (para fins de plotagem)
        nx_end, ny_end = mx + 30*nx, my + 30*ny  # O número 30 determina o comprimento dos vetores normais na plotagem

        # Desenha o vetor normal
        cv.arrowedLine(image_with_lines_and_normals, (int(mx), int(my)), (int(nx_end), int(ny_end)), (0, 0, 255), 1)

    # Mostra a imagem com linhas e vetores normais
    plt.imshow(cv.cvtColor(image_with_lines_and_normals, cv.COLOR_BGR2RGB))
    plt.show()


In [2]:
def gaussian_pyramid(image):
    i0 = image.copy()
    gpi0 = [i0.astype(float)]
    i0 = np.array(i0)
    kernel_size = 13
    (x,y) = np.meshgrid(np.linspace(-1,1,kernel_size), np.linspace(-1,1,kernel_size))
    sigma = 1.0
    gaussian_blur = np.exp(-(x**2+y**2)/(2*sigma**2))
    gaussian_blur = gaussian_blur/np.sum(gaussian_blur)
    for i in range(4):
        #gaussian filter applied in each color channel
        i0r = conv2(i0[:,:,0], gaussian_blur, mode='same')
        i0g = conv2(i0[:,:,1], gaussian_blur, mode='same')
        i0b = conv2(i0[:,:,2], gaussian_blur, mode='same')
        i0 = np.stack((i0r,i0g,i0b), axis=-1)
        
        #down sampling: resizing
        w = int(i0.shape[1]*0.5)
        h = int(i0.shape[0]*0.5)
        dim = (w,h)
        i0 = cv.resize(i0, dim, interpolation = cv.INTER_AREA)
        print(i0.shape)
        #save to array for further processing
        gpi0.append(i0)
    
        plt.subplot(2,2,i+1)
        
        plt.imshow(i0/255)
        cv.imshow("scale", i0)
        cv.waitKey(0)
    
    cv.destroyAllWindows()
    return gpi0

In [ ]:
def gp_detector(gpi0):
    for gpi in gpi0:
        estimate_i1d(gpi)
    

Descreva o procedimento para detecção de cantos de Harris e explique qual é a função da "Resposta de Haris" no procedimento

O método de detecção de cantos de Harris começa com a definição do tensor de estrutra da imagem. Depois de encontrá-lo, é definida uma janela de análise em que veremos se há ou não há corners nela. É calculada da seguinte forma: det(T) - k(Traço(T))², definindo k como 0.05, temos 3 ranges de valores para a resposta de harris.
Se a resposta for pequeno, então não há features de interesse na região
Se a resposta for < 0, então há edges na região
Se a resposta for grande, então há corner na região